This is the Basic Code for image generation


Adding frongted and UI features

In [ ]:
import torch
import gradio as gr
import random
import PIL.Image

Loading the Stable  Diffusion XL Pipeline

In [ ]:
# Load Stable Diffusion XL Pipeline
from diffusers import StableDiffusionXLPipeline
pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    variant="fp16" if torch.cuda.is_available() else None,
)
pipe = pipe.to("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Image generation function
def generate_image(prompt, style, aspect_ratio, negative_prompt):
    # Adjust prompt with selected style
    style_descriptions = {
        "Realistic": "highly detailed, photorealistic, ultra sharp",
        "Anime": "anime style, cel shading, vibrant colors",
        "Cyberpunk": "futuristic, neon lighting, cyberpunk aesthetics",
        "Sketch": "black and white, pencil sketch, line art",
        "Oil Painting": "oil painting style, brush strokes, canvas texture"
    }
    styled_prompt = f"{prompt}, {style_descriptions.get(style, '')}"

    width, height = {
        "1:1": (768, 768),
        "16:9": (1024, 576),
        "9:16": (576, 1024)
    }.get(aspect_ratio, (768, 768))

    # Run model
    image = pipe(
        prompt=styled_prompt,
        negative_prompt=negative_prompt,
        width=width,
        height=height,
        guidance_scale=7.5,
        num_inference_steps=30
    ).images[0]

    return image

In [ ]:
# Enhance prompt
def enhance_prompt(prompt):
    return f"{prompt}, extremely detailed, trending on ArtStation, masterpiece"

# Simulate login (not secure, just frontend mock)
def simulate_login(username, password):
    if username and password:
        return f"Welcome, {username}!", gr.update(visible=True)
    return "Invalid credentials.", gr.update(visible=False)

In [ ]:
# Build Gradio interface
with gr.Blocks(css="footer {visibility: hidden}") as demo:
    gr.Markdown("## 🎨 AI Text-to-Image Generator - SDXL Edition")

    with gr.Tab("🖼️ Generate Image"):
        with gr.Row():
            with gr.Column(scale=2):
                prompt = gr.Textbox(label="Prompt", placeholder="e.g., A wizard standing on a cliff...")
                enhance_btn = gr.Button("✨ Enhance Prompt")
                style = gr.Dropdown(["Realistic", "Anime", "Cyberpunk", "Sketch", "Oil Painting"], label="Style", value="Realistic")
                aspect = gr.Radio(["1:1", "16:9", "9:16"], label="Aspect Ratio", value="1:1")
                neg_prompt = gr.Textbox(label="Negative Prompt (optional)", placeholder="e.g., blurry, distorted, low quality")
                generate_btn = gr.Button("Generate 🔥")

            with gr.Column(scale=3):
                output_img = gr.Image(label="Generated Image", type="pil")
                download_btn = gr.Button("📥 Download Image")
                history_gallery = gr.Gallery(label="🕓 Generation History", show_label=True, columns=3)

    # with gr.Tab("📋 Login"):
    #     username = gr.Textbox(label="Username")
    #     password = gr.Textbox(label="Password", type="password")
    #     login_btn = gr.Button("🔐 Login")
    #     welcome_text = gr.Textbox(label="Status", interactive=False)
    #     login_panel = gr.Group(visible=False)

    #     login_btn.click(fn=simulate_login, inputs=[username, password], outputs=[welcome_text, login_panel])

    # with login_panel:
    #     gr.Markdown("✅ You are now logged in. Start generating awesome images!")

    # Event bindings
    enhance_btn.click(fn=enhance_prompt, inputs=prompt, outputs=prompt)

    image_list = []

    def generate_and_save(prompt, style, aspect, neg_prompt):
        img = generate_image(prompt, style, aspect, neg_prompt)
        image_list.append(img)
        return img, image_list

    generate_btn.click(fn=generate_and_save, inputs=[prompt, style, aspect, neg_prompt], outputs=[output_img, history_gallery])

    # Allow download (for full functionality, you could create a file link if needed)
    def download_image(img):
        path = "generated_image.png"
        img.save(path)
        return path

    download_btn.click(fn=download_image, inputs=output_img, outputs=gr.File(label="Download Link"))

# Launch app
demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5154509402c3792581.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
